<a href="https://colab.research.google.com/github/StevenVuong/MSc_Project/blob/master/v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install dependencies
!pip install deepbrain; # semi-colon to hide the output
!pip install pydicom;


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import pydicom
import pickle
from deepbrain import Extractor
from sklearn.model_selection import train_test_split
import nibabel as nb

In [3]:
from google.colab import drive

# mount google drive into google colab
drive.mount('/content/gdrive')

os.chdir('/content')

# go to where the data is
print (os.listdir())
os.chdir('gdrive/My Drive/msc_project')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
['.config', 'gdrive', 'sample_data']


In [0]:
# https://www.kaggle.com/sentdex/first-pass-through-data-w-3d-convnet
patients_file_dir = 'T1_SAG_SIEMEN_3T_CLEAN'

patients = os.listdir(patients_file_dir) # get all patients ID's in scan
patient_df = pd.read_csv('T1_SAG_SIEMEN_3T_CLEAN_5_29_2019.csv') # get dataframe too to cross reference

# Map GenCohort to regular PD and Controls
patient_df['Group'] = patient_df['Group'].replace({'GenCohort PD':'PD', 'GenCohort Unaff':'Control'})

# map control to 0 and pd to 1
patient_df['Group'] = patient_df['Group'].replace({'Control':0, 'PD':1})

**BUILD FUNCTION TO LOOP THROUGH DATA AND FIND WHAT IS THERE (EMPTY) AND WHAT IS NOT, THEN MODIFY THE DATA FRAME ACCORDINGLY!**

In [53]:
cwd = os.getcwd()
print (cwd)

def get_grappa_dir(path):
  # get the file ending with 'GRAPPA', would need to accomodate this for grappa also
  returning_path = None
  for next_path in os.listdir(path):
    if (next_path.split("_")[-1] == 'GRAPPA'): # for the t1 weighted
      returning_path = next_path
      return returning_path

def get_dcm_s(path):
  # get the path beginning with S, so doesn't clash with GZ File
  for next_path in os.listdir(path):
    if (next_path[0] == 'S'):
      return next_path

def get_img_no(path):
  # get the image identification numberm any image will do for this so take first
  image_number = None
  for image_file in os.listdir(path):
    image_number = int(image_file.split("_")[-1][1:-4]) # index to get the ID
   
  return image_number

def filename_sort(filename):
    
    # split by underlines and delimiter
    split_line = filename.split("_")
    int_return = int(split_line[-3])
    
    return int_return
  
def get_slices(patient):
  '''For any given patient, returns the slices for them'''
  try:
    # label = patient_df.get_value(patient, 'Subject') # cannot go by patient, must get the ID
    path = cwd + '/' + patients_file_dir + '/' + str(patient) # get to the GRAPPA 
    path = path + '/' + get_grappa_dir(path)
    path = path + '/' + os.listdir(path)[-1] # get the most recent scan for patient
    path = path + '/' + get_dcm_s(path)

    print ("PATH IS: %s " % path)
    # get information related around the image
    image_number = get_img_no(path)
    print ("1")
    print (image_number) # some image numbers are none..
    image_row = patient_df_sample.loc[patient_df_sample['Image Data ID'] == image_number] # relate to df
    print ("2")
    image_sex = image_row.Sex.values[0]
    print ("3")
    image_group = image_row.Group.values[0]
    print ("4")
    image_age = image_row.Age.values[0]

    # create image object and append to total info
    image_info = [image_number, image_sex, image_group, image_age]

    # print ("Sex: %s, Age: %s, Group: %s " % (image_sex, image_age, image_group))

    # get files and sort them in order
    dcm_files = os.listdir(path)
    print ("DCM FILES: " + len(dcm_files))
    if (len(dcm_files)==0):
      print ("DCM FILE FOR PATIENT %d IS EMPTY" % patient)
      
    dcm_files = sorted(dcm_files, key=lambda filename: filename_sort(filename)) # some have length 3

    slices = []
    # loop through slices and build the array
    for dcm_file in dcm_files:
      path_to_file = path + '/' + dcm_file
      slices.append(pydicom.read_file(path_to_file).pixel_array)
    slices = np.array(slices)[15:175, :, :]
    
    return slices, image_info
 
  except Exception as e:
    print ("No File Found: %s" % str(e))

/content/gdrive/My Drive/msc_project


In [0]:
def process_slice(total_slice):
  # deal with mixed slice information
  slices = total_slice[0]
  
  # transform into axial view
  slice_axial = slices.transpose((1,2,0))
  
  # initialise skull stripper
  ext = Extractor()

  # get probability of part of image being brain tissue or not
  prob = ext.run(slice_axial)
  mask = prob > 1e-3 # mask can be obtained as:
  slice_axial[~mask] = 0 # apply mask
  
  slice_axial = slice_axial[30:230, 30:230, :] # trim blank ones
  
  # flip images and add to total processed arrays
  flipped_slices = [np.flip(sl,1) for sl in slice_axial]
  
  return slice_axial, flipped_slices, total_slice[1]

In [0]:
import keras

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=10, dim=(200,200,160), n_channels=1,
                 n_classes=2, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization (modified for flipped slices)
        X = np.empty((2*self.batch_size, *self.dim, self.n_channels))
        y = np.empty((2*self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            
            try:
              # get and process sample
              slices, image_info = get_slices(ID) # grab slices for a patient
              print ("GOT THE SLICE!")
              slice_axial, flipped_slices, sl_info = process_slice([slices, image_info]) # works

              # Store sample
              j = 2*i
              X[j,] = slice_axial
              X[j+1,] = flipped_slices # flipped version
              ######################
              # Store class
              y[j] = self.labels[ID]
              y[j+1] = self.labels[ID]
              
            except Exception as e:
              print ("Ayyah we have an error: %s" % str(e))

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [0]:
# need to format data into the right method.. 
patient_df_sample = patient_df.sample(50)
patient_df_sample_id = patient_df_sample['Subject'].values

# split to train and test set
train_set, validation_set = train_test_split(patient_df_sample_id, test_size=0.2)

labels = pd.Series(patient_df_sample.Group.values,index=patient_df_sample.Subject).to_dict()

# build dictionary and populate
partition = {}
partition['train'] = train_set
partition['validation'] = validation_set

In [0]:
from keras.models import Sequential

# Parameters
params = {'dim': (200,200,160),
          'batch_size': 10,
          'n_classes': 2,
          'n_channels': 1,
          'shuffle': True}

# Generators
training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['validation'], labels, **params)

In [0]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils import np_utils, generic_utils
from keras.layers import LeakyReLU

In [11]:
# Design model
model = Sequential()

model.add(Convolution3D(filters=8, kernel_size=2, strides=1, padding='same', input_shape=(200,200,160,1))) # or should activation be linear?
model.add(LeakyReLU(alpha=0.01)) # set to 0.01
model.add(MaxPooling3D())

model.add(Convolution3D(filters=16, kernel_size=2, strides=1, padding='same'))
model.add(LeakyReLU(alpha=0.01)) 
model.add(MaxPooling3D())

model.add(Convolution3D(filters=32, kernel_size=3, strides=1, padding='same'))
model.add(LeakyReLU(alpha=0.01))
model.add(MaxPooling3D())

model.add(Convolution3D(filters=64, kernel_size=3, strides=1, padding='same'))
model.add(LeakyReLU(alpha=0.01))
model.add(MaxPooling3D())

model.add(Convolution3D(filters=128, kernel_size=4, strides=1, padding='same'))
model.add(LeakyReLU(alpha=0.01))
model.add(MaxPooling3D())

model.add(Convolution3D(filters=256, kernel_size=4, strides=1, padding='same'))
model.add(LeakyReLU(alpha=0.01))
model.add(MaxPooling3D())

model.add(Flatten())
model.add(Dense(512))
model.add(LeakyReLU(alpha=0.01))

model.add(Dense(2, activation='softmax'))

model.compile(optimizer=Adam(lr=0.00005), loss='categorical_crossentropy',metrics = ['accuracy']) # metrics=['categorical_accuracy']


Instructions for updating:
Colocations handled automatically by placer.


In [54]:
# Train model on dataset
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=False,
                    workers=1)

Epoch 1/1
PATH IS: /content/gdrive/My Drive/msc_project/T1_SAG_SIEMEN_3T_CLEAN/3833/MPRAGE_GRAPPA/2012-11-27_11_22_36.0/S184026 
PATH IS: /content/gdrive/My Drive/msc_project/T1_SAG_SIEMEN_3T_CLEAN/50901/MPRAGE_GRAPPA/2014-12-17_10_44_05.0/S259039 
1
None
12

NoneNo File Found: index 0 is out of bounds for axis 0 with size 0

2Ayyah No file found: 'NoneType' object is not iterable

No File Found: index 0 is out of bounds for axis 0 with size 0
Ayyah No file found: 'NoneType' object is not iterable
PATH IS: /content/gdrive/My Drive/msc_project/T1_SAG_SIEMEN_3T_CLEAN/3123/MPRAGE_GRAPPA/2012-06-13_09_37_20.0/S169307 
PATH IS: /content/gdrive/My Drive/msc_project/T1_SAG_SIEMEN_3T_CLEAN/4035/MPRAGE_GRAPPA/2015-01-20_11_08_05.0/S250448 
1
None
2
No File Found: index 0 is out of bounds for axis 0 with size 0
Ayyah No file found: 'NoneType' object is not iterable
1
337653
2
3
4
No File Found: must be str, not int
Ayyah No file found: 'NoneType' object is not iterable
PATH IS: /content/gdrive/M

IndexError: ignored